In [1]:
import os
os.chdir('../')

In [2]:
import pandas as pd
import torch
import torch.nn as nn
from torchvision.transforms import Compose, ToTensor, RandomHorizontalFlip, Normalize, Resize, RandomRotation
import numpy as np
from torch.utils.data import DataLoader
from modules.deep_pixel_wise.Dataset import PixWiseDataset
from modules.deep_pixel_wise.Model import DeePixBiS
from modules.deep_pixel_wise.Loss import PixWiseBCELoss
from modules.deep_pixel_wise.Metrics import predict, test_accuracy, test_loss
from modules.deep_pixel_wise.Trainer import Trainer

In [ ]:
train_data = pd.read_csv('.\data\\path_files\\train_data_celeb_nuaa.csv')
test_data = pd.read_csv('.\data\\path_files\\test_data_celeb_nuaa.csv')
val_data = pd.read_csv('.\data\\path_files\\val_data_celeb_nuaa.csv')

In [ ]:
train_data_shuffled = train_data.sample(frac=1)
train_data_shuffled.head()

In [3]:
torch.cuda.empty_cache()

In [4]:
model = DeePixBiS()
# model.load_state_dict(torch.load('/content/drive/MyDrive/FPT MSE/Capstone Project/Anti_Spoof_DPW/DeePixBiS.pth'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

loss_fn = PixWiseBCELoss()

opt = torch.optim.Adam(model.parameters(), lr=0.0001)

train_tfms = Compose([Resize([224, 224]),
                      RandomHorizontalFlip(),
                      RandomRotation(10),
                      ToTensor(),
                    #   Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
                      Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

test_tfms = Compose([Resize([224, 224]),
                     ToTensor(),
                    #  Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
                     Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_dataset = PixWiseDataset('.\data\\path_files\\train_data_celeb_nuaa.csv', transform=train_tfms)
train_ds = train_dataset.dataset()

val_dataset = PixWiseDataset('.\data\\path_files\\val_data_celeb_nuaa.csv', transform=test_tfms)
val_ds = val_dataset.dataset()

batch_size = 16
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=0, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size, shuffle=True, num_workers=0, pin_memory=True)

trainer = Trainer(train_dl, val_dl, model, 5, opt, loss_fn, device)

print('Training Beginning\n')
trainer.fit()

print('\nTraining Complete')
torch.save(model.state_dict(), './models/DeePixBiS/DeePixBiS_celeb_nuaa_260123.pth')

c:\KhoiNXM\Workspace\python_venv\face_recognition_system\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


cuda
Training Beginning


Epoch (1/5)
----------------------------------


c:\KhoiNXM\Workspace\python_venv\face_recognition_system\lib\site-packages\torch\nn\functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Loss : 0.692446231842041
Loss : 0.7081700563430786
Loss : 0.675376296043396
Loss : 0.599183201789856
Loss : 0.6736254692077637
Loss : 0.623955249786377
Loss : 0.588654637336731
Loss : 0.5497281551361084
Loss : 0.5760930180549622
Loss : 0.5453912019729614
Loss : 0.6515114307403564
Loss : 0.5268553495407104
Loss : 0.4775044918060303
Loss : 0.4508039355278015
Loss : 0.500619113445282
Loss : 0.47558730840682983
Loss : 0.5101454854011536
Loss : 0.4464538097381592
Loss : 0.47487056255340576
Loss : 0.5162633657455444
Loss : 0.4001259505748749
Loss : 0.3983888030052185
Loss : 0.5717832446098328
Loss : 0.361125648021698
Loss : 0.3923700451850891
Loss : 0.36822760105133057
Loss : 0.45132946968078613
Loss : 0.43708837032318115
Loss : 0.35730814933776855
Loss : 0.35720309615135193
Loss : 0.3217101991176605
Loss : 0.3503861427307129
Loss : 0.4788060784339905
Loss : 0.5505707263946533
Loss : 0.3017981946468353
Loss : 0.3290308117866516
Loss : 0.3227561116218567
Loss : 0.3438206613063812
Loss : 0.279

OutOfMemoryError: CUDA out of memory. Tried to allocate 38.00 MiB (GPU 0; 4.00 GiB total capacity; 3.26 GiB already allocated; 0 bytes free; 3.41 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF